In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from scipy.optimize import fsolve

In [9]:
#constants
hbc = 197.326967883
pi = np.pi
m_N = 939./hbc                 # Nucleon mass in fm^-1
m_omega = 783./hbc             # Omega meson mass in fm^-1
mw2 = m_omega**2
m_sigma = 550./hbc             # Sigma meson mass in fm^-1
ms2 = m_sigma**2
nb_sat = 0.153
mq = 350./hbc
b = 0.59
g=2

In [10]:
#
num_nB = 2000
nB0 = 0.153 #saturation density in fm^-3
nB_min = 1e-07 # in fm^-3
nB_max = 4.0 # in fm^-3
#
nB       = np.logspace(np.log10(nB_min), np.log10(nB_max), num_nB)
epsilon  = np.array([0. for n in range(0, num_nB)])
E0       = np.array([0. for n in range(0, num_nB)])
E0_coupl = np.array([0. for n in range(0, num_nB)])
m_Ns     = np.array([0. for n in range(0, num_nB)])
m_Nsapp  = np.array([0. for n in range(0, num_nB)])
m_NsI    = np.array([0. for n in range(0, num_nB)])
P        = np.array([0. for n in range(0, num_nB)])
n0       = np.array([0. for n in range(0, num_nB)])
gm2      = np.array([0. for n in range(0,1)])
#

In [11]:
#
def fmns(mns,*dat_gS2):
    kF  = dat_gS2[0]
    gS2 = dat_gS2[1]
    EF = np.sqrt(kF**2 + mns**2)
    ns = (g*mns/(2*pi**2)) * ( kF*EF - mns**2*np.log((kF+EF)/mns) )
    func = mns - (m_N - gS2*ns)
    return func
#
def mstar(kF,m_guess,gS2):
    dat_gS2 = (kF,gS2)
    mstar = fsolve(fmns, m_guess, args = dat_gS2)
    mnstar = mstar[0]
    return mnstar

In [19]:
def energy(kF,gS2,gW2):
    nb = g*kF**3/3/pi**2
    dat_gS2 = (kF,gS2)
    if kF < 1:
        m_guess = m_N
    else:
        m_guess = 0.1*m_N
    mns = fsolve(fmns, m_guess, args = dat_gS2)
    mnstar = mns[0]
    EF = np.sqrt(kF**2 + mnstar**2)
    ns = (g*mnstar/(2*pi**2)) * ( kF*EF - mnstar**2*np.log((kF+EF)/mnstar) )
    ener = 0.5*gW2*nb**2 + 0.5*gS2*ns**2 \
         + (g/(8*pi**2)) * (2*kF*EF**3 - mnstar**2 * kF*EF \
         - mnstar**4*np.log((kF+EF)/mnstar))    
    return ener

def pressure(kF,gS2,gW2):
    nb = g*kF**3/3/pi**2
    dat_gS2 = (kF,gS2)
    if kF < 1:
        m_guess = m_N
    else:
        m_guess = 0.1*m_N
    mns = fsolve(fmns, m_guess, args = dat_gS2)
    mnstar = mns[0]
    EF = np.sqrt(kF**2 + mnstar**2)
    ns = (g*mnstar/(2*pi**2)) * ( kF*EF - mnstar**2*np.log((kF+EF)/mnstar) )
    pres = 0.5*gW2*nb**2 - 0.5*gS2*ns**2 \
         + (1/(g*8*pi**2)) * (((2/3)*kF**3 - mnstar**2 * kF)*EF \
         + mnstar**4*np.log((kF+EF)/mnstar))
#    press_ex =
    return pres

In [20]:
nb = nb_sat
def coupls(gm2,*data):
    kF = data[0]
    gS2 = gm2[0]
    gW2 = gm2[1]
    delkF = kF/500.
    delen = energy(kF+delkF,gS2,gW2) - energy(kF-delkF,gS2,gW2)
    der_en = delen/2./delkF - g*kF**2/pi**2 * energy(kF,gS2,gW2)/nb
    print('* gS2, gW2 = ',gS2,gW2)
    print('* e, der_en, P = ',(energy(kF,gS2,gW2)/nb - m_N)*hbc,\
          der_en, pressure(kF,gS2,gW2))
    print(' ')
    FeP = np.empty((2))
    FeP[0] = der_en
    FeP[1] = energy(kF,gS2,gW2)/nb - m_N + 15.8/hbc #pressure(kF,gS2,gW2,mnstar) - 0.
    return FeP

In [27]:
target_energy = -15.8
tolerance = 1e-2  # Define a tolerância para a condição de parada
nb = nb_sat
kF = ((3*pi**2*nb)/g)**(1/3)
mnstar = m_N #510./hbc
#you can put any value for gS2 and gW2
gS2 = 2 
gW2 = 2
while True:
    data = (kF)
    couplGuess = np.array([gS2, gW2])
    coupl = fsolve(coupls, couplGuess, args=data)
    current_energy = (energy(kF, coupl[0], coupl[1]) / nb - m_N) * hbc

    print('*** e & p = ', current_energy, pressure(kF, coupl[0], coupl[1]))

    if abs(current_energy - target_energy) < tolerance:
        break

    gS2 = gS2 + 0.1 * (gS2 - coupl[0])
    gW2 = gW2 + 0.1 * (gW2 - coupl[1])
    print('gS2:', coupl[0], 'gW2:', coupl[1])
    print(' ')

print('Final constants: gS2 =', coupl[0], 'gW2 =', coupl[1])

gS2_final = coupl[0]
gW2_final = coupl[1]



* gS2, gW2 =  2 2
* e, der_en, P =  22.5672051509866 0.028862313617487034 0.004020657210123337
 
* gS2, gW2 =  2.0 2.0
* e, der_en, P =  22.5672051509866 0.028862313617487034 0.004020657210123337
 
* gS2, gW2 =  2.0 2.0
* e, der_en, P =  22.5672051509866 0.028862313617487034 0.004020657210123337
 
* gS2, gW2 =  2.0000000298023224 2.0
* e, der_en, P =  22.567204723198557 0.028862312889286423 0.004020656883279402
 
* gS2, gW2 =  2.0 2.0000000298023224
* e, der_en, P =  22.567205600867762 0.028862314414341395 0.0040206575589446176
 
* gS2, gW2 =  41.46810670757825 36.98824481742974
* e, der_en, P =  115.80931509530645 0.642690490660673 0.25154671443322224
 
* gS2, gW2 =  6.861926482374406 3.351837898086555
* e, der_en, P =  -26.171427994554037 -0.05048674002251419 -0.03223757273335314
 
* gS2, gW2 =  13.871856436548224 10.282319198163146
* e, der_en, P =  -17.85898319080933 -0.011924560830094633 -0.018517474245970188
 
* gS2, gW2 =  16.359327812473015 12.68616603865846
* e, der_en, P =  -